In [15]:
import pandas as pd
import numpy as np
import glob
import csv

In [2]:
lager = pd.read_excel('temp/GEWERBE - Rechnung - Lager.xlsx',
                      header=2,
                      dtype={
                          "Jahr" : str,
                      })

/usr/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


remove the stuff below in the Lager file

In [3]:
lager = lager.iloc[0:lager['Jahr'].last_valid_index()+1]

In [4]:
lager = lager.convert_dtypes({
    'verkauft' : np.float64,
})

Load Lager and remove all rows where Artikelnr is not set.

Ignore rows that have an invalid value in the Artikelnr column

In [44]:
lager_agg = lager[lager['eingestellt'].isnull()].groupby('Artikelnr', dropna=True).agg(
    available=('Stk', 'sum'),
    price=('verkauft', 'max'))

In [7]:
lager[lager['Artikelnr'] == 'DEU2015002320']['verkauft'].max()

3.5

Load Coin Database

In [8]:
coin_db = pd.read_excel('Münzdatenbank.xlsx',
                        dtype={'groupid': str})

Generate the Picpath

In [9]:
coin_db['picpath'] = coin_db['Artikelnr'] + '0.jpg'

Append the required 0 and 2 to the end of the Artikelnr

In [10]:
coin_db_2 = coin_db.copy()
coin_db_2['Artikelnr'] = coin_db['Artikelnr'] + '2'
coin_db_2['rate'] = "20"

coin_db['Artikelnr'] = coin_db['Artikelnr'] + '0'
coin_db['rate'] = "0"

coin_db = pd.concat([coin_db, coin_db_2])

In [11]:
coin_db = coin_db.set_index('Artikelnr')

In [12]:
coins_in_shop = lager_agg.join(coin_db, how="right")
coins_in_shop.loc[coins_in_shop['available'].notna(), 'CMD'] = ''
coins_in_shop['available'] = coins_in_shop['available'].fillna(0)

In [77]:
coin_db_missing = lager_agg.join(coin_db, how="left")
coin_db_missing = coin_db_missing.loc[coin_db_missing['name'].isna(), ['available', 'price']
                   ].join(lager.set_index('Artikelnr')['Münzbezeichnung'],
                          how='left').reset_index().groupby('Münzbezeichnung').first()
coin_db_missing.to_excel("fehlt.xlsx")
print("Fehlende Münzen in der Münzdatenbank:")
coin_db_missing

Fehlende Münzen in der Münzdatenbank:


,Artikelnr,available,price
Münzbezeichnung,,,
10 Euro Silber 2020 PP 75 J. Ende 2. Weltkrieg,MAl2020010100,1,69.0
10 Euro Silber 2020 PP UNESCO Verbotene Stadt,FRA2020010350,2,64.0
2 Euro SM Deutschland 2016 Dresdner Zwinger J Rolle,DEU2016002240,1,58.0
2 Euro SM Frankreich 2012 Abbe Pierre Rolle,FRA201200220,1,92.0
2 Euro SM Frankreich 2014 AIDS Rolle,FRA201400260,1,72.0
...,...,...,...
3 Euro SM 2020 30 J. Volksabstimmung zur Unabhängigkeit Rolle,SLO2020003200,10,5.0
30 g Silber China 2020 Panda,CHI2020001100,2,26.0
KMS 2004 1ct - 2 Euro unc lose,DEU2004388190,1,9.0


In [13]:
coins_in_shop.to_csv('gesamt.csv',
                             sep='\t',
                             encoding="cp1252",
                             line_terminator='\r\n',
                             quoting=csv.QUOTE_MINIMAL,
                             quotechar='\'',
                             index=True,
                             index_label='realid'
                            )